<h2 align='center'>Codebasics ML Course: ML Flow Tutorial</h2>

In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [5]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

### Track Experiments

In [6]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [7]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [ ]:
reports

In [8]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [9]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/01/21 10:33:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/706566654211917845/runs/2e00f7a0ae3d469eaf0fcf508ee707e6
🧪 View experiment at: http://localhost:5000/#/experiments/706566654211917845


2025/01/21 10:33:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/706566654211917845/runs/3c2c98e752c64706aae6af88508dfab1
🧪 View experiment at: http://localhost:5000/#/experiments/706566654211917845


2025/01/21 10:33:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/706566654211917845/runs/862023ca4af24edfb2d3871e75275157
🧪 View experiment at: http://localhost:5000/#/experiments/706566654211917845


2025/01/21 10:33:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/706566654211917845/runs/67c6c360691f4aab8648643adea5c827
🧪 View experiment at: http://localhost:5000/#/experiments/706566654211917845


### Register the Model

In [10]:
model_name = 'XGB-Smote'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model_name'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Please type RunID 67c6c360691f4aab8648643adea5c827


Successfully registered model 'XGB-Smote'.
2025/01/21 10:34:00 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/706566654211917845/runs/67c6c360691f4aab8648643adea5c827
🧪 View experiment at: http://localhost:5000/#/experiments/706566654211917845


Created version '1' of model 'XGB-Smote'.


### Load the Model

In [26]:
model_version = 1
model_uri = f"models:/{model_name}@challenger"
# model_uri = ""

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

OSError: No such file or directory: 'C:\Users\pooja\Desktop\Udemy-MLOps\mlflow_practice\mlflow_dagshub_demo\mlruns\706566654211917845\67c6c360691f4aab8648643adea5c827\artifacts\model_name\.'

In [16]:
mlflow.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1737435840305, description='', last_updated_timestamp=1737435840340, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1737435840340, current_stage='None', description='', last_updated_timestamp=1737435840340, name='XGB-Smote', run_id='67c6c360691f4aab8648643adea5c827', run_link='', source='file:///C:/Users/pooja/Desktop/Udemy-MLOps/mlflow_practice/mlflow_dagshub_demo/mlruns/706566654211917845/67c6c360691f4aab8648643adea5c827/artifacts/model_name', status='READY', status_message='', tags={}, user_id='', version='1'>], name='XGB-Smote', tags={}>]

### Transition the Model to Production

In [25]:
current_model_uri = f"models:/{model_name}@challenger"
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1737439546933, current_stage='None', description='', last_updated_timestamp=1737439546933, name='anomaly-detection-prod', run_id='67c6c360691f4aab8648643adea5c827', run_link='', source='models:/XGB-Smote/1', status='READY', status_message='', tags={}, user_id='', version='1'>

In [27]:
model_version = 1
prod_model_uri = f"models:/{production_model_name}@champion"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

OSError: No such file or directory: 'C:\Users\pooja\Desktop\Udemy-MLOps\mlflow_practice\mlflow_dagshub_demo\mlruns\706566654211917845\67c6c360691f4aab8648643adea5c827\artifacts\model_name\.'

Please refer to following to learn more about model registry

https://mlflow.org/docs/latest/model-registry.html#model-registry-workflows to learn 